In [ ]:
import os

In [ ]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
LABEL_MAP_NAME = 'label_map.pbtxt'

TRAIN_PATH = '/content/drive/MyDrive/Tensorflow/hack'

datasets for yolo

In [ ]:
class_ids = [
    "window",
    "empty",
    "filled",
]
class_mapping = dict(zip([1,2,3], class_ids))

In [ ]:
import os
import io
import glob
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from collections import namedtuple


def split(df, group):
   data = namedtuple('data', ['file_name', 'object'])
   gb = df.groupby(group)
   return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
    # with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.file_name)), 'rb') as fid:
    #     encoded_jpg = fid.read()
    # encoded_jpg_io = io.BytesIO(encoded_jpg)
    # image = Image.open(encoded_jpg_io)
    # width, height = image.size
    # image = tf.convert_to_tensor(np.array(image))

    filename = group.file_name.encode('utf8')
    image_format = b'jpg'
    bbox = []
    classes = []

    for index, row in group.object.iterrows():
        bbox.append([row['xmin'], row['ymin'], row['w'], row['h']])
        classes.append(row['class'])


    return 1, bbox, classes, group.file_name

In [ ]:
def get_data_dicts(df):
  images = []
  classes = []
  boxes = []

  g = split(df, 'file_name')
  for group in g:
      tf_example = create_tf_example(group, TRAIN_PATH)
      boxes.append(tf_example[1])
      classes.append(tf_example[2])
      images.append(os.path.join(TRAIN_PATH, tf_example[3]))
      # image = tf.io.read_file()
      # image = tf.image.decode_jpeg(image, channels=3)
      # image = tf.cast(image, tf.float32)
      # images.append(image)

  boxes = tf.ragged.constant(boxes)
  classes = tf.ragged.constant(classes)
  images = tf.ragged.constant(images)



  data = tf.data.Dataset.from_tensor_slices((images, classes, boxes))
  # out = {'images': images,
  #        'bounding_boxes': {
  #           "boxes" :boxes,
  #           'classes' : classes,
  #           }
  #        }
  return data




In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Tensorflow/hack/data.csv')
df['w'] = df['xmax'] - df['xmin']
df['h'] = df['ymax'] - df['ymin']
df = df.drop(['xmax', 'ymax'], axis=1)

train_example = df.head(int(len(df)*(80/100)))
test_example = df.tail(int(len(df)*(20/100)))

train_data = get_data_dicts(train_example)
test_data = get_data_dicts(test_example)

In [ ]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    return image


def load_dataset(image_path, classes, bbox):
    # Read Image
    image = load_image(image_path)
    bounding_boxes = {
        "classes": tf.cast(classes, dtype=tf.float32),
        "boxes": bbox,
    }
    return {"images": tf.cast(image, tf.float32), "bounding_boxes": bounding_boxes}


In [ ]:
import keras
!pip install -q keras_cv
import keras_cv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.1/803.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 944.9/944.9 kB 48.8 MB/s eta 0:00:00
Using TensorFlow backend


In [ ]:
import tensorflow as tf

In [ ]:
augmenter = keras.Sequential(
    layers=[
        keras_cv.layers.RandomFlip(mode="horizontal", bounding_box_format="xywh"),
        keras_cv.layers.RandomShear(
            x_factor=0.2, y_factor=0.2, bounding_box_format="xywh"
        ),
        keras_cv.layers.JitteredResize(
            target_size=(640, 640), scale_factor=(0.75, 1.3), bounding_box_format="xywh"
        ),
    ]
)

resizing = keras_cv.layers.JitteredResize(
    target_size=(640, 640),
    scale_factor=(0.75, 1.3),
    bounding_box_format="xywh",
)


In [ ]:
BATCH_SIZE = 2
train_ds = train_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(BATCH_SIZE * 4)
train_ds = train_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)
train_ds = train_ds.map(augmenter, num_parallel_calls=tf.data.AUTOTUNE)


val_ds = test_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.shuffle(BATCH_SIZE * 4)
val_ds = val_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)
val_ds = val_ds.map(resizing, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
# def visualize_dataset(inputs, value_range, rows, cols, bounding_box_format):
#     inputs = next(iter(inputs.take(1)))
#     images, bounding_boxes = inputs["images"], inputs["bounding_boxes"]
#     keras_cv.visualization.plot_bounding_box_gallery(
#         images,
#         value_range=value_range,
#         rows=1,
#         cols=cols,
#         y_true=bounding_boxes,
#         scale=5,
#         font_scale=0.7,
#         bounding_box_format=bounding_box_format,
#         class_mapping=class_mapping,
#     )


# visualize_dataset(
#     val_ds, bounding_box_format="xywh", value_range=(0, 255), rows=2, cols=2
# )

In [ ]:
def dict_to_tuple(inputs):
    # return inputs["images"], {
    #     'classes':inputs["bounding_boxes"]['classes'].to_tensor(),
    #       "boxes": inputs["bounding_boxes"]['boxes'].to_tensor()
    #       }
    return inputs["images"], inputs["bounding_boxes"]
    # return inputs["images"], keras_cv.bounding_box.to_dense(
    #     inputs["bounding_boxes"], max_boxes=200
    # )



train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

val_ds = val_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
train_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(2, 640, 640, 3), dtype=tf.float32, name=None), {'classes': RaggedTensorSpec(TensorShape([2, None]), tf.float32, 1, tf.int64), 'boxes': RaggedTensorSpec(TensorShape([2, None, None]), tf.float32, 1, tf.int64)})>

Формат датасета



bounding_boxes = {
    # num_boxes may be a Ragged dimension
    'boxes': Tensor(shape=[batch, num_boxes, 4]),
    'classes': Tensor(shape=[batch, num_boxes])
}


{"images": images, "bounding_boxes": bounding_boxes}


In [ ]:


backbone = keras_cv.models.YOLOV8Backbone.from_preset(
    "yolo_v8_s_backbone_coco"  # We will use yolov8 small backbone with coco weights
)
yolo = keras_cv.models.YOLOV8Detector(
    num_classes=len(class_mapping),
    bounding_box_format="xyxy",
    backbone=backbone,
    fpn_depth=1,
)

In [ ]:
type(yolo)

keras_cv.src.models.object_detection.yolo_v8.yolo_v8_detector.YOLOV8Detector

In [ ]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    global_clipnorm=15.0,
)

yolo.compile(
    optimizer=optimizer, classification_loss="binary_crossentropy", box_loss="ciou"
)

In [ ]:
class EvaluateCOCOMetricsCallback(keras.callbacks.Callback):
    def __init__(self, save_path):
        super().__init__()
        self.save_path = save_path

    def on_epoch_end(self, epoch, logs):
        self.model.save(self.save_path)
        return logs

In [ ]:
yolo.fit(
    train_ds,
    epochs=50,
    validation_data=val_ds,
    callbacks=[EvaluateCOCOMetricsCallback("/content/drive/MyDrive/Colab Notebooks/Keras/model2.keras")],
)

Epoch 1/50
98/98 [==============================] - 248s 2s/step - loss: 1214.5371 - box_loss: 1.9675 - class_loss: 1212.5698 - val_loss: 22320.1191 - val_box_loss: 2.6608 - val_class_loss: 22317.4570
Epoch 2/50
98/98 [==============================] - 195s 2s/step - loss: 35.7498 - box_loss: 2.1438 - class_loss: 33.6060 - val_loss: 22.7258 - val_box_loss: 2.4849 - val_class_loss: 20.2409
Epoch 3/50
98/98 [==============================] - 197s 2s/step - loss: 3.5075 - box_loss: 1.8964 - class_loss: 1.6111 - val_loss: 4.5246 - val_box_loss: 3.0574 - val_class_loss: 1.4672
Epoch 4/50
98/98 [==============================] - 197s 2s/step - loss: 3.0748 - box_loss: 1.9967 - class_loss: 1.0781 - val_loss: 4.5653 - val_box_loss: 2.4769 - val_class_loss: 2.0884
Epoch 5/50
98/98 [==============================] - 213s 2s/step - loss: 2.3139 - box_loss: 1.4683 - class_loss: 0.8455 - val_loss: 3.8388 - val_box_loss: 2.5563 - val_class_loss: 1.2825
Epoch 6/50
98/98 [=============================

In [ ]:
yolo.save("/content/drive/MyDrive/Colab Notebooks/Keras/model2.keras")

In [ ]:
def visualize_detections(model, dataset, bounding_box_format):
    images, y_true = next(iter(dataset.take(1)))
    y_pred = model.predict(images)
    y_pred = keras_cv.bounding_box.to_ragged(y_pred)
    print(y_pred)
    keras_cv.visualization.plot_bounding_box_gallery(
        images,
        value_range=(0, 255),
        bounding_box_format=bounding_box_format,
        y_true=y_true,
        y_pred=y_pred,
        scale=5,
        rows=1,
        cols=1,
        show=True,
        font_scale=0.7,
        class_mapping=class_mapping,
    )


visualize_detections(yolo, dataset=train_ds, bounding_box_format="xywh")

In [ ]:
yolo.summary()